In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
import math    
import folium
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist
import sys
from haversine import haversine
from shapely.geometry import LineString
from shapely.ops import split, linemerge
import krippendorff

In [2]:
import sys
sys.path.append('../')

from src.cluster import *
from src.colors import *
from src.map import *
from src.compare import *

## Krippendorff's alpha

In [46]:
#read in chicago validation labels
chicago_labels = gpd.read_file('../data/round-3/chicago-validation-labels-round3-230927.geojson')
chicago_labels = chicago_labels.drop(columns=['time_created'])
chicago_labels.head()

,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,correct,severity,temporary,tag_list,description,geometry
0,Devon Snyder,7958,1064,Addison,SurfaceProblem,41.976437,-87.979591,Vnk-r0opTiyjIyISxOMRUQ,None,1,f,cracks,None,POINT (-87.97959 41.97644)
1,Devon Snyder,7961,1064,Addison,SurfaceProblem,41.976780,-87.979561,w2tdzPaq9Rl7drhiXjPrvw,None,1,f,cracks,None,POINT (-87.97956 41.97678)
2,Devon Snyder,7965,1064,Addison,SurfaceProblem,41.977058,-87.979546,tfCdM8I4Mq5HZTZyleqk_g,None,1,f,height difference,None,POINT (-87.97955 41.97706)
3,Devon Snyder,7966,1064,Addison,SurfaceProblem,41.977795,-87.979553,dq_qwSfjMcwFLphaV8qzRw,None,1,f,cracks,None,POINT (-87.97955 41.97779)
4,Devon Snyder,7967,1064,Addison,SurfaceProblem,41.978218,-87.979507,MMoUtn6D9XovUCMh9vfZgQ,None,2,f,"cracks,grass",None,POINT (-87.97951 41.97822)


In [47]:
#count the cluster numbers for each label type
def count_clusters(labels, label_type_id):
    df = cluster_label_type_at_index(labels,label_type_id)[2]
    df["clustered"] = df.duplicated(subset="cluster_id", keep=False)
    df1 = df[df['clustered'] == True]
    return df1.cluster_id.nunique()

In [48]:
cluster_count_chicago = {}
for i in [2,4,5]:
    clusters =  count_clusters(chicago_labels, i)
    cluster_count_chicago[label_types[i]] = clusters

In [50]:
cluster_count_chicago

{'Problem': 141, 'SurfaceProblem': 109, 'Obstacle': 25}

In [52]:
#count the numbers for eanch label type
chicago_labels.label_type.value_counts()

SurfaceProblem    318
Obstacle           67
Name: label_type, dtype: int64

In [53]:
#read in chicago streets
chicago_streets = gpd.read_file("../data/round-3/chicago-streets-230927.geojson")

In [54]:
#split line into segments of 30 ft
def split_line_at_intervals(line, interval=20):
    distance = 0
    segments = []

    while distance < line.length:
        sub_line = line.interpolate(distance)
        next_distance = distance + interval
        next_sub_line = line.interpolate(next_distance)
        segment = LineString([sub_line, next_sub_line])
        segments.append(segment)
        distance = next_distance

    return segments

def split_geodataframe(geodf, local_crs, interval=20):
    
    geodf = geodf.to_crs(local_crs)
    
    new_rows = []

    for index, row in geodf.iterrows():
        line = row.geometry
        splitted_lines = split_line_at_intervals(line, interval)

        for segment_id, segment in enumerate(splitted_lines):
            new_row = row.copy()
            new_row.geometry = segment
            new_row['segment_id'] = segment_id
            new_rows.append(new_row)

    df = gpd.GeoDataFrame(new_rows, columns=geodf.columns.append(pd.Index(['segment_id'])))
    #add a column that combines street_edge_id and segment_id
    df['street_edge_segment_id'] =  df['street_edge_id'].astype(str) + '_' +  df['segment_id'].astype(str)
    df.crs = local_crs
    return df

In [55]:
#obtain cluster to street segment mapping
def cluster_to_street_segment_mapping(df,streets_seg):
     # Group by cluster_id and combine all geometries in each group into a MultiPoint
     df_grouped = df.groupby('cluster_id')['geometry'].apply(lambda x: x.unary_union)
     # Calculate centroid of each group
     centroids = df_grouped.centroid
     # Convert centroids to GeoDataFrame
     cluster_centroids = gpd.GeoDataFrame(geometry=centroids).reset_index()
     cluster_centroids.crs = {'init' :'epsg:4326'}
     cluster_centroids_ft = cluster_centroids.to_crs(epsg=streets_seg.crs.to_epsg())
     #map gdf_centroids to chicago_split, delete index_right column
     cluster_centeroids_street = cluster_centroids_ft.sjoin_nearest(streets_seg, distance_col='distance')
     cluster_centeroids_street = cluster_centeroids_street.drop(columns=['index_right'])
     #select only cluster_id and street_edge_segment_id
     cluster_centeroids_street = cluster_centeroids_street[['cluster_id','street_edge_segment_id']]
     #if there are multiple street_edge_segment_ids for a cluster_id, select the first one
     cluster_centeroids_street = cluster_centeroids_street.groupby('cluster_id').first().reset_index()
     return cluster_centeroids_street

In [56]:
#construct agreement table
def irr_table (df,street_seg):
    mikey = df[df['username'] == 'mikey']
    devon = df[df['username'] == 'Devon Snyder']
    irr_mikey = mikey.merge(street_seg, on='street_edge_segment_id', how='right').fillna(0)
    irr_devon = devon.merge(street_seg, on='street_edge_segment_id', how='right').fillna(0)
    #replace usernames with 1
    irr_mikey  = irr_mikey .replace('mikey',1)
    irr_devon  = irr_devon .replace('Devon Snyder',1)
    #group by street_edge_segment_id and count the number of label_ids
    irr_mikey = irr_mikey.groupby('street_edge_segment_id').sum()
    irr_devon = irr_devon.groupby('street_edge_segment_id').sum()
    irr_mikey = irr_mikey[['username']].reset_index()
    irr_devon = irr_devon[['username']].reset_index()
    irr_mikey = irr_mikey.rename(columns={'username':'mikey'})
    irr_devon = irr_devon.rename(columns={'username':'devon'})
    irr_mikey_devon = irr_mikey.merge(irr_devon, on='street_edge_segment_id', how='left').fillna(0)
    irr_mikey_devon = irr_mikey_devon.drop(columns=['street_edge_segment_id'])
    
    return irr_mikey_devon

In [57]:
def krippendorff_alpha(labels,label_type_id, streets, local_crs):
    #clustering by label type
    label_type = cluster_label_type_at_index(labels,label_type_id)[2]
    #split streets into segments
    street_seg = split_geodataframe(streets, local_crs, interval=30)
    #map cluster_id to street_edge_segment_id
    mapping = cluster_to_street_segment_mapping(label_type,street_seg)
    #update the street_edge_segment_id with the one closest to the cluster centroid
    label_type = label_type.merge(mapping, on='cluster_id', how='left')
    #select only the columns we need
    df = label_type[['username','cluster_id','street_edge_segment_id','label_id']]
    #construct agreement table
    irr = irr_table(df,street_seg)
    data_matrix = irr.T.to_numpy()
    alpha = krippendorff.alpha(data_matrix,level_of_measurement='ratio')

    return alpha.round(3)
    

In [58]:
alpha_chicago = {}
for i in [2,4,5]:
    alpha = krippendorff_alpha(chicago_labels,i, chicago_streets, 3435)
    alpha_chicago[label_types[i]] = alpha

In [60]:
#30 ft segments
alpha_chicago

{'Problem': 0.701, 'SurfaceProblem': 0.67, 'Obstacle': 0.772}

### surface problem and obstacle for severity level > = 4

In [61]:
#take out label type = surfaceproblem and obstacle
chicago_problems= chicago_labels[(chicago_labels['label_type'] == 'SurfaceProblem') | (chicago_labels['label_type'] == 'Obstacle') ]
#select only severity >=4
chicago_problems = chicago_problems[chicago_problems['severity'] >= 4]
chicago_problems

,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,correct,severity,temporary,tag_list,description,geometry
77,Devon Snyder,8048,1282,Milton,Obstacle,41.889133,-88.082375,dO0AodCUIut3ZeK6n2Gi1w,None,5,t,construction,None,POINT (-88.08237 41.88913)
78,Devon Snyder,8049,1282,Milton,Obstacle,41.889187,-88.082359,_ePnYNRLnQBFoQDgcFevGQ,None,5,t,construction,None,POINT (-88.08236 41.88919)
79,Devon Snyder,8050,1280,Milton,Obstacle,41.889458,-88.082375,b30ZCfKfuoEL0Z-jAe86Ig,None,5,t,construction,None,POINT (-88.08237 41.88946)
80,Devon Snyder,8052,1281,Milton,SurfaceProblem,41.889332,-88.082367,BHk6WiNy3M_wTR5kDhUbxQ,None,5,t,construction,None,POINT (-88.08237 41.88933)
81,Devon Snyder,8053,1282,Milton,SurfaceProblem,41.889160,-88.082367,_ePnYNRLnQBFoQDgcFevGQ,None,5,t,construction,None,POINT (-88.08237 41.88916)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,mikey,8726,1043,Bloomingdale,SurfaceProblem,41.905552,-88.062744,X_iqwvKs3knSe4Z75P-iyg,None,4,f,"sand/gravel,very broken,cracks,uneven/slanted",None,POINT (-88.06274 41.90555)
302,mikey,8727,1043,Bloomingdale,SurfaceProblem,41.905582,-88.062752,9bS_PR4HDLAyM_5oRjAcfQ,None,4,f,"very broken,cracks,uneven/slanted",None,POINT (-88.06275 41.90558)
329,mikey,8769,855,Downers Grove North,SurfaceProblem,41.749256,-87.973930,HeJubvJG_cLZXSoVtIb0tw,None,4,f,"sand/gravel,construction",None,POINT (-87.97393 41.74926)
360,mikey,8806,1032,Downers Grove North,SurfaceProblem,41.773804,-87.973244,qdSZu8koPeXWAjH2egYsqA,None,4,f,height difference,None,POINT (-87.97324 41.77380)


In [62]:
alpha_chicago_problems = {}
for i in [2,4,5]:
    alpha = krippendorff_alpha(chicago_problems,i, chicago_streets, 3435)
    alpha_chicago_problems[label_types[i]] = alpha

In [64]:
alpha_chicago_problems

{'Problem': 0.78, 'SurfaceProblem': 0.307, 'Obstacle': 0.786}

### surface problem and obstacle for severity level != 1

In [38]:
#take out label type = surfaceproblem and obstacle
chicago_problems= chicago_labels[(chicago_labels['label_type'] == 'SurfaceProblem') | (chicago_labels['label_type'] == 'Obstacle') ]
#take out severity ==1
chicago_problems = chicago_problems[chicago_problems['severity'] != 1]
chicago_problems

,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,correct,severity,temporary,tag_list,description,geometry
4,Devon Snyder,7967,1064,Addison,SurfaceProblem,41.978218,-87.979507,MMoUtn6D9XovUCMh9vfZgQ,None,2,f,"cracks,grass",None,POINT (-87.97951 41.97822)
5,Devon Snyder,7968,1064,Addison,SurfaceProblem,41.978207,-87.979507,MMoUtn6D9XovUCMh9vfZgQ,None,2,f,"cracks,grass",None,POINT (-87.97951 41.97821)
6,Devon Snyder,7969,1064,Addison,SurfaceProblem,41.980171,-87.979698,2QvdcRwkYT5XLaFbUQCjRA,None,2,f,height difference,None,POINT (-87.97970 41.98017)
9,Devon Snyder,7972,1130,Addison,SurfaceProblem,41.980820,-87.979683,fUes_2WwIb9F1_HEwVdWzw,None,2,f,height difference,None,POINT (-87.97968 41.98082)
14,Devon Snyder,7977,1043,Bloomingdale,Obstacle,41.905193,-88.063087,Nsyr_c_N9FsvS5r1gsH1vw,None,2,f,vegetation,None,POINT (-88.06309 41.90519)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,mikey,8817,1032,Downers Grove North,SurfaceProblem,41.773621,-87.971169,oAW7qUAPqjC8tE7xT_1UMQ,None,3,f,cracks,None,POINT (-87.97117 41.77362)
370,mikey,8819,1032,Downers Grove North,SurfaceProblem,41.773846,-87.970528,Ml4UM6L2eh7ZgOfCIRNeBA,None,4,f,"uneven/slanted,height difference,utility panel",None,POINT (-87.97053 41.77385)
371,mikey,8820,1032,Downers Grove North,SurfaceProblem,41.773621,-87.970558,uLzd3hfcBRbzCHw4vHuzDg,None,2,f,"height difference,utility panel",None,POINT (-87.97056 41.77362)
375,mikey,8825,1128,Downers Grove North,SurfaceProblem,41.773895,-87.968437,lGBc6lmptDLsVpnZcvDDqw,None,2,f,"cracks,grass",None,POINT (-87.96844 41.77390)


In [39]:
alpha_chicago_problems = {}
for i in [2,4,5]:
    alpha = krippendorff_alpha(chicago_problems,i, chicago_streets, 3435)
    alpha_chicago_problems[label_types[i]] = alpha

In [40]:
alpha_chicago_problems

{'Problem': 0.607, 'SurfaceProblem': 0.454, 'Obstacle': 0.845}

## HTML

In [ ]:
for i in [0,1,2,4,5,7,8,9]:
    labels= map_clusters(chicago_labels,i)
    map = chicago_streets.explore(m = labels, tiles="CartoDb DarkMatter", color="#BF616A", style_kwds={"stroke":True,"opacity":0.4,"weight": 2},tooltip=["street_edge_id"])
    map.save("html/chicago_" + label_types[i]+ "_230608.html")



In [ ]:
for i in [2,4,5]:
    labels= map_clusters(chicago_problems,i)
    map = chicago_streets.explore(m = labels, tiles="CartoDb DarkMatter", color="#BF616A", style_kwds={"stroke":True,"opacity":0.4,"weight": 2},tooltip=["street_edge_id"])
    map.save("html/chicago_wo_sev1_"+label_types[i]+"_230608.html")